In [92]:
import pandas as pd
import numpy as np
train_titanic = pd.read_csv("/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/datasets/train.csv")
test_titanic = pd.read_csv("/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/datasets/test.csv")
train_titanic.head()
test_titanic.shape

(4277, 13)

In [93]:
test_titanic.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


>+ I shuffle my data with the desire to improve the model's learning ability

In [94]:
from sklearn.utils import shuffle
train_titanic = shuffle(train_titanic)
test_titanic = shuffle(test_titanic)

In [95]:
train_titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8693 entries, 4045 to 7346
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 959.3+ KB


In [96]:
train_titanic.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

### **Features scalling data**

In [97]:
col_names = "RoomService", "FoodCourt","ShoppingMall","Spa","VRDeck"
roomservice_ix, foodcourt_ix, shopping_ix, spa_ix, vrCheck_ix = [
    train_titanic.columns.get_loc(c) for c in col_names] # get the column indices
print(roomservice_ix, foodcourt_ix, shopping_ix, spa_ix, vrCheck_ix)

7 8 9 10 11


In [98]:
import math
from sklearn.base import BaseEstimator, TransformerMixin
attribs_spending_money = [7,8,9,10,11]
# roomservice_ix, foodcourt_ix, shopping_ix, spa_ix, vrCheck_ix = 7,8,9,10,11
class CombineSpendingMoneyOfPerson(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        array = np.zeros(len(X)) # mang mot cot toan so 0
        for ele in range(len(X)):
            for i in attribs_spending_money:
                if ~math.isnan(X[ele][i]):
                    array[ele] += (X[ele][i])
        return np.c_[X,array]

# function for add feature ["spen_money_of_person"]
def feature_scalling_dataset(train, test, drop=None):
    attr_adder = CombineSpendingMoneyOfPerson()
    # Dataset with added columns ["spending_money_of_person"]
    train_titanic_extra_attribs = attr_adder.transform(train.values)
    test_titanic_extra_attribs = attr_adder.transform(test.values)
    train_titanic_extra_attribs = pd.DataFrame(train_titanic_extra_attribs,
                            columns=list(train) + ["spending_money_of_person"],
                            index= train.index)
    test_titanic_extra_attribs = pd.DataFrame(test_titanic_extra_attribs,
                            columns=list(test) + ["spending_money_of_person"],
                            index= test.index)
    return train_titanic_extra_attribs, test_titanic_extra_attribs
train_titanic_1, test_titanic_1 = feature_scalling_dataset(train_titanic, test_titanic)
train_titanic_1.isnull().sum()

PassengerId                   0
HomePlanet                  201
CryoSleep                   217
Cabin                       199
Destination                 182
Age                         179
VIP                         203
RoomService                 181
FoodCourt                   183
ShoppingMall                208
Spa                         183
VRDeck                      188
Name                        200
Transported                   0
spending_money_of_person    908
dtype: int64

In [99]:
# Split columns ["Cabin"] to ["Deck","Side"]
def deck_side_encode(data):
    split_cabin = pd.DataFrame(list(map(lambda x: str(x).split("/"), data.Cabin)))
    new_data = data.copy()
    new_data["Side"] = split_cabin[2]
    new_data["Deck"] = split_cabin[0]
    new_data = new_data.drop("Cabin", axis=1)
    return new_data

In [100]:
# for handle categories feature
from sklearn.preprocessing import LabelEncoder

def encode_labels(datasets):
  label_encoder = LabelEncoder()
  for column in datasets:
        dtype = datasets[column].dtype
        if dtype not in ['int64','float64']:
            datasets[column] = label_encoder.fit_transform(datasets[column])
  return datasets

In [101]:
attribs_num = ["Age","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck","spending_money_of_person"]
attribs_cat = ["HomePlanet", "Destination", "Cabin"]
attribs_cat_bool = ["CryoSleep", "VIP"]

### **Handle Data**

In [102]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

def prepare_data(train, test, drop=None):
    train, test = feature_scalling_dataset(train, test)
    # delete columns Name and PassengerId
    train = train.drop(["Name", "PassengerId"], axis=1)
    test = test.drop(["Name", "PassengerId"], axis=1)
    # missing value
    imputer = SimpleImputer(strategy="mean")
    imputer_cat_bool = SimpleImputer(strategy="most_frequent")

    train[attribs_num] = imputer.fit_transform(train[attribs_num])
    test[attribs_num] = imputer.fit_transform(test[attribs_num])

    train[attribs_cat_bool] = imputer_cat_bool.fit_transform(train[attribs_cat_bool])
    test[attribs_cat_bool] = imputer_cat_bool.fit_transform(test[attribs_cat_bool])

    train[attribs_cat] = imputer_cat_bool.fit_transform(train[attribs_cat])
    test[attribs_cat] = imputer_cat_bool.fit_transform(test[attribs_cat])
    
    #OnehotEncoder
    train = pd.get_dummies(deck_side_encode(train), columns=["HomePlanet","Destination","Deck","Side"])
    test = pd.get_dummies(deck_side_encode(test), columns=["HomePlanet","Destination","Deck","Side"])
    # implement LabelEncoder()
    train_encoded = encode_labels(train)
    test_encoded = encode_labels(test)
    # test_encoded_titan_encoder["Transported"] = test_encoded_titan_encoder["Transported"].astype(int)
    return train_encoded, test_encoded 

In [103]:
train_prep, test_prep = prepare_data(train_titanic, test_titanic)

In [104]:
train_prep.isnull().sum()

CryoSleep                    0
Age                          0
VIP                          0
RoomService                  0
FoodCourt                    0
ShoppingMall                 0
Spa                          0
VRDeck                       0
Transported                  0
spending_money_of_person     0
HomePlanet_Earth             0
HomePlanet_Europa            0
HomePlanet_Mars              0
Destination_55 Cancri e      0
Destination_PSO J318.5-22    0
Destination_TRAPPIST-1e      0
Deck_A                       0
Deck_B                       0
Deck_C                       0
Deck_D                       0
Deck_E                       0
Deck_F                       0
Deck_G                       0
Deck_T                       0
Side_P                       0
Side_S                       0
dtype: int64

In [105]:
train_prep.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,spending_money_of_person,...,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Side_P,Side_S
4045,0,3.0,0,0.000000,0.0,0.0,0.0,0.0,1,0.000000,...,0,0,0,0,0,0,1,0,1,0
3718,0,3.0,0,0.000000,0.0,0.0,0.0,0.0,0,0.000000,...,0,0,0,0,0,1,0,0,1,0
5501,1,14.0,0,0.000000,0.0,0.0,0.0,0.0,1,0.000000,...,0,0,0,0,1,0,0,0,1,0
878,0,38.0,0,55.000000,741.0,0.0,0.0,13.0,1,809.000000,...,0,0,0,0,0,0,1,0,0,1
6028,1,18.0,0,224.687617,0.0,0.0,0.0,0.0,1,1484.601541,...,0,0,0,0,0,0,1,0,0,1


### **Create and evaluate models**

In [106]:
X_train = np.array(train_prep.drop("Transported", axis=1))
y_train = np.array(train_prep.Transported)
X_test = np.array(test_prep)
X_train.shape, X_test.shape, y_train.shape

((8693, 25), (4277, 25), (8693,))

In [107]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
X_train_scaler = std_scaler.fit_transform(X_train)
X_test_scaler = std_scaler.fit_transform(X_test)

In [108]:
from random import random
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

linear_classifier = LogisticRegression(random_state=42, solver='lbfgs')
cross_val_score(linear_classifier, X_train, y_train, cv=3, scoring="accuracy").mean()

/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/tuanenv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/tuanenv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternati

0.78177825136448

#### Using DecisonTreeClassification


In [109]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_grid = {
    'criterion': ["gini","log_loss", "entropy"],
    'max_leaf_nodes': list(range(2, 100)),
    'max_depth':randint(low=1, high=50),
    "min_samples_split":randint(low=3, high = 5),
}

tree_classifier = DecisionTreeClassifier(random_state=42)
tree_grid_search = RandomizedSearchCV(tree_classifier, param_distributions=param_grid,
                                            n_iter=50, cv= 10, scoring="accuracy", random_state=42, verbose=1)

In [110]:
tree_grid_search.fit(X_train_scaler, y_train)

Fitting 10 folds for each of 50 candidates, totalling 500 fits


RandomizedSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=50,
                   param_distributions={'criterion': ['gini', 'log_loss',
                                                      'entropy'],
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f519420c820>,
                                        'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8,
                                                           9, 10, 11, 12, 13,
                                                           14, 15, 16, 17, 18,
                                                           19, 20, 21, 22, 23,
                                                           24, 25, 26, 27, 28,
                                                           29, 30, 31, ...],
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f519425b610>},
                   random_state=42, scoring='accuracy', verbose=1)

In [111]:
tree_grid_search.best_estimator_

DecisionTreeClassifier(max_depth=33, max_leaf_nodes=77, min_samples_split=4,
                       random_state=42)

In [112]:
tree_grid_search.best_score_

0.7876433474862109

In [113]:
finish_model_decison_tree_classifier = tree_grid_search.best_estimator_

### Evalue model with f1_score

In [114]:
from sklearn.metrics import f1_score
y_pred = finish_model_decison_tree_classifier.predict(X_train_scaler)
f1_score(y_train, y_pred)

0.8234511803169677

In [115]:
from sklearn.metrics import classification_report
evalue = classification_report(y_train, y_pred)
print(evalue)

              precision    recall  f1-score   support

           0       0.84      0.78      0.81      4315
           1       0.80      0.85      0.82      4378

    accuracy                           0.82      8693
   macro avg       0.82      0.82      0.82      8693
weighted avg       0.82      0.82      0.82      8693



#### **GradientBoostingClassifier**

In [116]:
from sklearn.ensemble import GradientBoostingClassifier
boosting_param_grid = {
    'loss':["deviance", "exponential"],
    'learning_rate':[0.03, 0.07, 0.1, 0.2],
    'n_estimators': [101, 151, 201, 301, 501, 1000],
    'criterion':["friedman_mse", "squared_error", "mse"],
}

gradient_boosting_classifier = GradientBoostingClassifier()
boosting_grid = RandomizedSearchCV(gradient_boosting_classifier, param_distributions=boosting_param_grid,
                                    n_iter=10, cv= 5, scoring="accuracy", random_state=42, verbose=1)

In [117]:
boosting_grid.fit(X_train_scaler, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/tuanenv/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:542: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/tuanenv/lib/python3.8/site-packages/sklearn/tree/_classes.py:397: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/tuanenv/lib/python3.8/site-packages/sklearn/tree/_classes.py:397: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warnings.warn(
/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/tuanenv/lib/python3.8/site-packages/sklearn/tree/_classes.py:397: FutureWar

RandomizedSearchCV(cv=5, estimator=GradientBoostingClassifier(),
                   param_distributions={'criterion': ['friedman_mse',
                                                      'squared_error', 'mse'],
                                        'learning_rate': [0.03, 0.07, 0.1, 0.2],
                                        'loss': ['deviance', 'exponential'],
                                        'n_estimators': [101, 151, 201, 301,
                                                         501, 1000]},
                   random_state=42, scoring='accuracy', verbose=1)

In [118]:
boosting_grid.best_estimator_

GradientBoostingClassifier(learning_rate=0.07, loss='deviance',
                           n_estimators=101)

In [119]:
boosting_grid.best_score_

0.7922479024822143

In [120]:
boosting_model = boosting_grid.best_estimator_

In [121]:
y_pred_boosting = boosting_model.predict(X_train_scaler)
error = classification_report(y_train, y_pred_boosting)
print(error)

              precision    recall  f1-score   support

           0       0.83      0.77      0.80      4315
           1       0.79      0.85      0.82      4378

    accuracy                           0.81      8693
   macro avg       0.81      0.81      0.81      8693
weighted avg       0.81      0.81      0.81      8693



In [122]:
'''
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import NuSVC
from sklearn.metrics import accuracy_score

import xgboost as xgb

estimators = [
    ('xgb', xgb.XGBClassifier(max_depth=3, n_estimators=91, subsample=0.65, colsample_bytree=0.65, eval_metric='rmse')),
    ('tree', DecisionTreeClassifier(criterion='entropy', max_depth=16, max_leaf_nodes=63,min_samples_split=3, random_state=42)),
    ('nusvc', make_pipeline(StandardScaler(), NuSVC())),
    ('knn', KNeighborsClassifier(n_neighbors=38))
]
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
'''

"\nfrom sklearn.ensemble import StackingClassifier\nfrom sklearn.neighbors import KNeighborsClassifier\nfrom sklearn.svm import NuSVC\nfrom sklearn.metrics import accuracy_score\n\nimport xgboost as xgb\n\nestimators = [\n    ('xgb', xgb.XGBClassifier(max_depth=3, n_estimators=91, subsample=0.65, colsample_bytree=0.65, eval_metric='rmse')),\n    ('tree', DecisionTreeClassifier(criterion='entropy', max_depth=16, max_leaf_nodes=63,min_samples_split=3, random_state=42)),\n    ('nusvc', make_pipeline(StandardScaler(), NuSVC())),\n    ('knn', KNeighborsClassifier(n_neighbors=38))\n]\nclf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())\n"

In [123]:
#clf.fit(X_train_tf, y_train)

In [124]:
# y_pred_stacking = clf.predict(X_train_tf)

In [125]:
#err_stacking = classification_report(y_train, y_pred_stacking)
#print(err_stacking)

In [126]:
test_titanic_ = pd.read_csv("/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/datasets/test.csv")
test_titanic_.shape

(4277, 13)

In [127]:
# Result with boosting model
y1 = boosting_model.predict(X_test_scaler)

In [128]:
result = pd.DataFrame(y1, columns=["Transported"])
result.shape

(4277, 1)

In [129]:
test_id = test_titanic_["PassengerId"]
submission = pd.DataFrame({
    "PassengerId":test_id,
    "Transported":y1.astype(bool),
})
display(submission.head(10))

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,True
3,0021_01,False
4,0023_01,False
5,0027_01,False
6,0029_01,True
7,0032_01,False
8,0032_02,True
9,0033_01,True


In [130]:
submission.to_csv("/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/result boosting/submission.csv", index=False)

In [131]:
from sklearn.ensemble import RandomForestClassifier
forest_param_grid = {
    'n_estimators':[100,200,300,400,700,1000],
    'criterion':["gini", "entropy", "log_loss"],
    'max_depth':[3,4,5,6,7,10],
}
forest_classifier = RandomForestClassifier(random_state=42)
forest_grid = RandomizedSearchCV(forest_classifier, param_distributions=forest_param_grid,
                                 n_iter=25, cv = 10, verbose=2, random_state=42)

In [132]:
forest_grid.fit(X_train_scaler, y_train)

Fitting 10 folds for each of 25 candidates, totalling 250 fits
[CV] END .criterion=log_loss, max_depth=3, n_estimators=1000; total time=   4.2s
[CV] END .criterion=log_loss, max_depth=3, n_estimators=1000; total time=   4.2s
[CV] END .criterion=log_loss, max_depth=3, n_estimators=1000; total time=   4.4s
[CV] END .criterion=log_loss, max_depth=3, n_estimators=1000; total time=   4.2s
[CV] END .criterion=log_loss, max_depth=3, n_estimators=1000; total time=   4.2s
[CV] END .criterion=log_loss, max_depth=3, n_estimators=1000; total time=   4.2s
[CV] END .criterion=log_loss, max_depth=3, n_estimators=1000; total time=   4.2s
[CV] END .criterion=log_loss, max_depth=3, n_estimators=1000; total time=   4.2s
[CV] END .criterion=log_loss, max_depth=3, n_estimators=1000; total time=   4.2s
[CV] END .criterion=log_loss, max_depth=3, n_estimators=1000; total time=   4.3s
[CV] END ......criterion=gini, max_depth=4, n_estimators=700; total time=   3.2s
[CV] END ......criterion=gini, max_depth=4, n_

RandomizedSearchCV(cv=10, estimator=RandomForestClassifier(random_state=42),
                   n_iter=25,
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': [3, 4, 5, 6, 7, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         700, 1000]},
                   random_state=42, verbose=2)

In [133]:
forest_grid.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=200,
                       random_state=42)

In [134]:
model_forest = forest_grid.best_estimator_

In [135]:
y_pred3 = model_forest.predict(X_train_scaler)
forest_error = classification_report(y_train, y_pred3)
print(forest_error)

              precision    recall  f1-score   support

           0       0.85      0.82      0.84      4315
           1       0.83      0.86      0.85      4378

    accuracy                           0.84      8693
   macro avg       0.84      0.84      0.84      8693
weighted avg       0.84      0.84      0.84      8693



In [136]:
y3 = model_forest.predict(X_test_scaler)
test_id = test_titanic_["PassengerId"]
submission1 = pd.DataFrame({
    "PassengerId":test_id,
    "Transported":y3.astype(bool),
})
display(submission.head())

submission.to_csv("/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/result with RandomForestClassifier/submission.csv", index=False)

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,True
3,0021_01,False
4,0023_01,False


In [137]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
forest_param_grid = {
    'n_estimators':[100,200,300,400,700,1000],
    'criterion':["gini", "entropy", "log_loss"],
    'max_depth':[3,4,5,6,7,10],
}
forest_classifier = RandomForestClassifier(random_state=42)
forest_grid_search = GridSearchCV(forest_classifier, param_grid=forest_param_grid, scoring="accuracy",
                            verbose=2)

In [138]:
forest_grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] END ......criterion=gini, max_depth=3, n_estimators=100; total time=   0.4s
[CV] END ......criterion=gini, max_depth=3, n_estimators=100; total time=   0.4s
[CV] END ......criterion=gini, max_depth=3, n_estimators=100; total time=   0.4s
[CV] END ......criterion=gini, max_depth=3, n_estimators=100; total time=   0.4s
[CV] END ......criterion=gini, max_depth=3, n_estimators=100; total time=   0.4s
[CV] END ......criterion=gini, max_depth=3, n_estimators=200; total time=   0.7s
[CV] END ......criterion=gini, max_depth=3, n_estimators=200; total time=   0.7s
[CV] END ......criterion=gini, max_depth=3, n_estimators=200; total time=   0.7s
[CV] END ......criterion=gini, max_depth=3, n_estimators=200; total time=   0.8s
[CV] END ......criterion=gini, max_depth=3, n_estimators=200; total time=   0.7s
[CV] END ......criterion=gini, max_depth=3, n_estimators=300; total time=   1.1s
[CV] END ......criterion=gini, max_depth=3, n_

GridSearchCV(estimator=RandomForestClassifier(random_state=42),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [3, 4, 5, 6, 7, 10],
                         'n_estimators': [100, 200, 300, 400, 700, 1000]},
             scoring='accuracy', verbose=2)

In [143]:
forest_grid_search.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=300,
                       random_state=42)

In [144]:
forest_grid_search.best_score_

0.7967346946878323

In [145]:
grisearch_forest_model = forest_grid_search.best_estimator_

In [148]:
grisearch_forest_model.fit(X_train_scaler, y_train)

RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=300,
                       random_state=42)

In [149]:
y_result = grisearch_forest_model.predict(X_train_scaler)
err = classification_report(y_train, y_result)
print(err)

              precision    recall  f1-score   support

           0       0.85      0.82      0.84      4315
           1       0.83      0.86      0.85      4378

    accuracy                           0.84      8693
   macro avg       0.84      0.84      0.84      8693
weighted avg       0.84      0.84      0.84      8693



In [151]:
y_result_1 = grisearch_forest_model.predict(X_test_scaler)
test_id = test_titanic_["PassengerId"]
submission1 = pd.DataFrame({
    "PassengerId":test_id,
    "Transported":y_result_1.astype(bool),
})
display(submission.head())

submission.to_csv("/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/result with gridsearchCV/submission.csv", index=False)

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,True
3,0021_01,False
4,0023_01,False
